#### zigbang 매물 데이터 저장
- pip install -y geohash2

In [2]:
import pymongo
import zigbang as zb
import pandas as pd

In [3]:
datas = zb.oneroom("마포구 합정동")

In [4]:
df = pd.DataFrame(datas)
df.head(2)

,section_type,item_id,images_thumbnail,sales_type,sales_title,deposit,rent,size_m2,공급면적,전용면적,...,is_zzim,status,service_type,tags,address1,address2,address3,manage_cost,reg_date,is_new
0,None,24371037,https://ic.zigbang.com/ic/items/24371037/1.jpg,월세,월세,2000,70,46.28,"{'m2': 46.28, 'p': '14'}","{'m2': 44.63, 'p': '13.5'}",...,False,True,빌라,[],서울시 마포구 합정동,None,None,0,2020-10-16T16:05:33+09:00,False
1,None,24019996,https://ic.zigbang.com/ic/items/24019996/1.jpg,전세,전세,18000,0,71.11,"{'m2': 71.11, 'p': '21.5'}","{'m2': 68.81, 'p': '20.8'}",...,False,True,빌라,[],서울시 마포구 합정동,None,None,1.5,2020-10-19T14:27:54+09:00,False


In [5]:
# server 연결
server = pymongo.MongoClient('mongodb://52.78.52.151:27017/')
db = server.zigbang

In [6]:
addrs = {
    "mongwon": "마포구 망원동",
    "hapjung": "마포구 합정동",
}

In [13]:
# 데이터 수집 후 저장
for collection_name, addr in addrs.items():
    collection = db[collection_name]
    datas = zb.oneroom(addr)
    collection.insert_many(datas) 
    # ids = collection.insert(datas) # pymongo version update (insert -> insert_many)
    # print(collection_name, addr, len(ids))

In [14]:
# 망원동에서 월세 50이하 보증금 1000에서 3000조건으로 검색
QUERY = {"rent": {"$lte": 50}, "deposit": {"$lte": 3000, "$gte": 1000}}
results = db["mongwon"].find(QUERY)
df = pd.DataFrame(results).tail()
columns = ["title", "service_type", "sales_type", "deposit", "rent", "size_m2", "floor", "building_floor", "address1", "manage_cost", "is_new"]
df[columns]

,title,service_type,sales_type,deposit,rent,size_m2,floor,building_floor,address1,manage_cost,is_new
7,망원역5분//망원시장옆//큰방,원룸,월세,1000,40,19.83,2,4,서울시 마포구 망원동,3,False
8,🧡💛망원역 도보5분거리 정말 저렴하고 넓은 방!🧡💛,원룸,월세,1000,40,23.14,2,3,서울시 마포구 망원동,3,True
9,✔⛧채광좋은 풀옵션원룸 내부깔끔⛧✔,원룸,월세,3000,40,23.14,3,4,서울시 마포구 망원동,5,False
10,✔⭐리모델링 첫입주 투룸✔망원시장 인근✔채광 좋아요♻⭐,빌라,월세,1000,50,42.98,반지하,2,서울시 마포구 망원동,1,True
11,⭐⭐베란다 있는 원룸⭐주택가 조용⭐마포구청역3분⭐⭐,원룸,월세,1000,40,19.83,5,6,서울시 마포구 망원동,3,False


In [15]:
# 컬렉션 삭제
for addr in addrs:
    print(addr)
    server.zigbang.drop_collection(addr)

mongwon
hapjung


In [16]:
# 데이터 베이스 삭제
server.drop_database("zigbang")